In [6]:
import torch
import dhg
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

class GCNConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, g: dhg.Graph) -> torch.Tensor:
        if not isinstance(g, dhg.Graph):
            raise TypeError(f"Expected dhg.Graph, got {type(g)}")  
        X = self.theta(X)
        X_ = g.smoothing_with_GCN(X)
        X_ = self.drop(self.act(X_))
        return X_

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.5):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.bn1 = torch.nn.BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = torch.nn.BatchNorm1d(hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)     
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)


    

In [7]:
def train(model, data, optimizer=None, criterion=None):
    model.train()

    # Initialize optimizer & loss function if not provided
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss()

    optimizer.zero_grad()
    out = model(data['x'], data['graph'])  # Forward pass
    
    # Compute loss
    loss = criterion(out[data['train_mask']], data['y'][data['train_mask']])
    
    # Backpropagation
    loss.backward()
    optimizer.step()

    # Compute accuracy
    pred = out.argmax(dim=1)  # Get predicted class indices
    correct = (pred[data['train_mask']] == data['y'][data['train_mask']]).sum().item()
    accuracy = correct / data['train_mask'].sum().item()

    return loss.item(), accuracy


def evaluate(model, data, criterion=None):
    model.eval()
    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        out = model(data['x'], data['graph'])  # Forward pass
        loss = criterion(out[data['test_mask']], data['y'][data['test_mask']])

        pred = out.argmax(dim=1)
        correct = (pred[data['test_mask']] == data['y'][data['test_mask']]).sum().item()
        accuracy = correct / data['test_mask'].sum().item()

    return loss.item(), accuracy


In [8]:
%run data.ipynb

PATH_TO_IMAGE = "E:\MINI_PROJECT\PAVIA_UNI\PaviaU.mat"
PATH_TO_LABEL = "E:\MINI_PROJECT\PAVIA_UNI\PaviaU_gt.mat"

data = DATA(PATH_TO_IMAGE, PATH_TO_LABEL)

Creating graph with 207400 nodes and 1653504 edges.


In [9]:
graph_data = data.get_data_for_gcn()

# Now, you can use it with your GCN model
x = graph_data['x']
graph = graph_data['graph']  # Use 'graph' instead of 'edge_index'
y = graph_data['y']

# Example Model Usage
model = GCN(input_dim=x.shape[1], hidden_dim=16, output_dim=y.max().item() + 1)

# Example Training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [10]:
num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, graph_data, optimizer, criterion)
    test_loss, test_acc = evaluate(model, graph_data, criterion)

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

E:\Anaconda\Lib\site-packages\dhg\structure\graphs\graph.py:633: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  self.cache["L_GCN"] = _tmp_g.D_v_neg_1_2.mm(_tmp_g.A).mm(_tmp_g.D_v_neg_1_2).clone().coalesce()


Epoch 1/50 - Train Loss: 2.4299, Train Acc: 0.0891 | Test Loss: 1.6059, Test Acc: 0.7928
Epoch 2/50 - Train Loss: 2.3547, Train Acc: 0.1670 | Test Loss: 2.0142, Test Acc: 0.2489
Epoch 3/50 - Train Loss: 2.2719, Train Acc: 0.2801 | Test Loss: 1.9648, Test Acc: 0.6990
Epoch 4/50 - Train Loss: 2.2067, Train Acc: 0.3701 | Test Loss: 1.9049, Test Acc: 0.7735
Epoch 5/50 - Train Loss: 2.1526, Train Acc: 0.4516 | Test Loss: 1.8120, Test Acc: 0.7785
Epoch 6/50 - Train Loss: 2.1024, Train Acc: 0.5234 | Test Loss: 1.7223, Test Acc: 0.7848
Epoch 7/50 - Train Loss: 2.0577, Train Acc: 0.5777 | Test Loss: 1.6417, Test Acc: 0.7916
Epoch 8/50 - Train Loss: 2.0139, Train Acc: 0.6258 | Test Loss: 1.5961, Test Acc: 0.7928
Epoch 9/50 - Train Loss: 1.9761, Train Acc: 0.6670 | Test Loss: 1.5784, Test Acc: 0.7928
Epoch 10/50 - Train Loss: 1.9440, Train Acc: 0.6925 | Test Loss: 1.5809, Test Acc: 0.7928
Epoch 11/50 - Train Loss: 1.9085, Train Acc: 0.7145 | Test Loss: 1.5980, Test Acc: 0.7928
Epoch 12/50 - Train